In [77]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import keys
import json
import datetime

In [2]:
url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'

In [59]:
req = requests.get(f'{url}&apikey={keys.ticketmaster}')

In [60]:
req.status_code

200

In [162]:
x = json.loads(req.text)

In [163]:
df = pd.DataFrame(x['_embedded']['events'])
df.columns

Index(['name', 'type', 'id', 'test', 'url', 'locale', 'images', 'sales',
       'dates', 'classifications', 'promoter', 'promoters', 'pleaseNote',
       'priceRanges', 'products', 'seatmap', 'ticketLimit', '_links',
       '_embedded', 'info', 'accessibility'],
      dtype='object')

In [164]:
df.drop(columns = ['id','images','pleaseNote','test','info', 'accessibility', 'locale','classifications', 'promoter',
                   'type','_links', 'seatmap', 'products', 'dates'],inplace=True)

In [165]:
#add binary sold out var
df.priceRanges.fillna(1, inplace=True)
df['sold_out'] = df.priceRanges.apply(lambda x : 1 if x == 1 else 0)

In [166]:
#unpack sales
# convert diff to feature
df.sales[1]['public']['startDateTime'][0:10]
df.sales[1]['public']['endDateTime'][0:10]

'2020-06-08'

In [167]:
#unpack sales
# presale feature
if df.sales[1]['presales']:
    print(1)

1


In [168]:
venue_df = pd.io.json.json_normalize(df['_embedded'].apply(lambda x: x['venues'][0]))

In [169]:
venue_df = venue_df[['name','postalCode','markets','city.name', 'state.name', 'state.stateCode', 'country.countryCode', 
                     'address.line1','location.longitude', 'location.latitude']]

In [170]:
#counts of markets or weighted dist?
venue_df.markets[0]

[{'name': 'New York/Tri-State Area', 'id': '35'},
 {'name': 'Northern New Jersey', 'id': '55'},
 {'name': 'Connecticut', 'id': '124'}]

In [175]:
venue_df.columns = ['venue_name', 'postalCode', 'markets', 'city.name', 'state.name',
       'state.stateCode', 'country.countryCode', 'address.line1',
       'location.longitude', 'location.latitude']

In [177]:
df.join(venue_df)

,name,url,sales,promoters,priceRanges,ticketLimit,_embedded,sold_out,venue_name,postalCode,markets,city.name,state.name,state.stateCode,country.countryCode,address.line1,location.longitude,location.latitude
0,New York Giants vs. New York Jets,https://www.ticketmaster.com/new-york-giants-v...,{'public': {'startDateTime': '2019-04-18T01:00...,"[{'id': '704', 'name': 'NFL EXHIBITION & PRESE...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is a six (6) ticket limit per ...,"{'venues': [{'name': 'MetLife Stadium', 'type'...",0,MetLife Stadium,07073,"[{'name': 'New York/Tri-State Area', 'id': '35...",East Rutherford,New Jersey,NJ,US,One MetLife Stadium Drive,-74.074493,40.81359
1,2020 CMA Fest,https://www.ticketmaster.com/2020-cma-fest-nas...,{'public': {'startDateTime': '2019-08-02T15:00...,"[{'id': '494', 'name': 'PROMOTED BY VENUE', 'd...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is an eight (8) ticket limit f...,"{'venues': [{'name': 'Nissan Stadium', 'type':...",0,Nissan Stadium,37213,"[{'name': ' Knoxville & More""', 'id': '31'}]",Nashville,Tennessee,TN,US,One Titans Way,-86.771413,36.166208
2,Baltimore Orioles vs. New York Yankees,https://www.ticketmaster.com/baltimore-orioles...,{'public': {'startDateTime': '2019-03-01T15:00...,"[{'id': '685', 'name': 'MLB REGULAR SEASON', '...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is an overall 14 ticket limit ...,{'venues': [{'name': 'Oriole Park At Camden Ya...,0,Oriole Park At Camden Yards,21201,"[{'name': ' DC and Maryland""', 'id': '47'}]",Baltimore,Maryland,MD,US,333 West Camden Street,-76.6222465,39.28196765
3,Eagles,https://www.ticketmaster.com/eagles-las-vegas-...,{'public': {'startDateTime': '2019-04-12T17:00...,"[{'id': '494', 'name': 'PROMOTED BY VENUE', 'd...",1,{'info': 'Ticket limits vary per price level. ...,"{'venues': [{'name': 'MGM Grand Garden Arena',...",1,MGM Grand Garden Arena,89109,"[{'name': 'Las Vegas', 'id': '14'}]",Las Vegas,Nevada,NV,US,3799 Las Vegas Blvd South,-115.1694962,36.1023376
4,San Francisco 49ers vs. Dallas Cowboys,https://www.ticketmaster.com/san-francisco-49e...,{'public': {'startDateTime': '2019-04-18T00:00...,"[{'id': '705', 'name': 'NFL REGULAR SEASON', '...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is an 8 ticket limit for this ...,"{'venues': [{'name': 'Levi's® Stadium', 'type'...",0,Levi's® Stadium,95054,"[{'name': 'N. California/N. Nevada', 'id': '41'}]",Santa Clara,California,CA,US,4900 Marie P. DeBartolo Way,-121.97034311,37.40367671
5,Kansas City Chiefs vs. Green Bay Packers,https://www.ticketmaster.com/kansas-city-chief...,{'public': {'startDateTime': '2019-04-18T15:00...,"[{'id': '705', 'name': 'NFL REGULAR SEASON', '...","[{'type': 'standard', 'currency': 'USD', 'min'...",NaN,"{'venues': [{'name': 'Arrowhead Stadium', 'typ...",0,Arrowhead Stadium,64129,"[{'name': 'Kansas City and More', 'id': '26'},...",Kansas City,Missouri,MO,US,1 Arrowhead Dr,-94.48417065,39.04921048
6,KoRn & Alice In Chains,http://concerts.livenation.com/event/1D0056554...,{'public': {'startDateTime': '2019-03-01T15:00...,"[{'id': '653', 'name': 'LIVE NATION MUSIC', 'd...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is a 8 ticket limit per person.'},{'venues': [{'name': 'Northwell Health at Jone...,0,Northwell Health at Jones Beach Theater,11793,"[{'name': 'New York/Tri-State Area', 'id': '35...",Wantagh,New York,NY,US,895 Bay Parkway,-73.5023618,40.6014755
7,Chicago Bears vs. Carolina Panthers,https://www.ticketmaster.com/chicago-bears-vs-...,{'public': {'startDateTime': '2019-04-18T01:30...,"[{'id': '704', 'name': 'NFL EXHIBITION & PRESE...","[{'type': 'standard', 'currency': 'USD', 'min'...",{'info': 'There is a twenty (20) ticket limit....,"{'venues': [{'name': 'Soldier Field', 'type': ...",0,Soldier Field,60605,"[{'name': 'Chicagoland and Northern Il', 'id':...",Chicago,Illinois,IL,US,1410 S. Museum Campus Drive,-87.61682143,41.8620